# Handwritten Digit Recognition App using the MNIST Dataset -- Convolutional Neural Networks (CNN)

Importing Libraries
- For this project, we will be using the Keras library which contains the MNIST dataset along with numpy and tensorflow libraries.
- Then, we will use the mnist.load_data() method to split train and test data in order to return training data and its labels and testing data and its labels.

In [1]:
import numpy as np
import tensorflow as tf
import keras

In [2]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()
print(x_train.shape, y_train.shape)

(60000, 28, 28) (60000,)


Data Preprocessing
- We will be tranforming image data from class vectors to binary class matrices in order to prepare it for our neural network.
- The CNN model will require one more dimension, therefore, reshaping the dimension of the dataset to (60000,28,28,1).

In [6]:
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
input_shape = (28, 28, 1)

# converting class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


Convolutional Neural Networks (CNN) Model
- We will be using the CNN model since our data is represented as grid structures and the CNN model will be very effective for image classification purpose of this application. 
- Then, we compile the model with the Adadelta optimizer.

In [7]:
batch_size = 128
num_classes = 10
epochs = 10
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))
model.compile(loss=keras.losses.categorical_crossentropy,optimizer=keras.optimizers.Adadelta(),metrics=['accuracy'])

Training the model
- We will train the model using the model_fit() function of Keras library, which will take the training data, validation data, epochs, and batch size.

In [8]:
hist = model.fit(x_train, y_train,batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(x_test, y_test))
print("The model has successfully trained")
model.save('mnist.h5')
print("Saving the model as mnist.h5")

Epoch 1/10
469/469 [==============================] - 63s 133ms/step - loss: 2.2678 - accuracy: 0.1572 - val_loss: 2.2134 - val_accuracy: 0.3810
Epoch 2/10
469/469 [==============================] - 61s 130ms/step - loss: 2.1769 - accuracy: 0.2977 - val_loss: 2.0977 - val_accuracy: 0.5976
Epoch 3/10
469/469 [==============================] - 60s 129ms/step - loss: 2.0557 - accuracy: 0.4372 - val_loss: 1.9419 - val_accuracy: 0.7134
Epoch 4/10
469/469 [==============================] - 61s 129ms/step - loss: 1.8927 - accuracy: 0.5449 - val_loss: 1.7328 - val_accuracy: 0.7725
Epoch 5/10
469/469 [==============================] - 61s 129ms/step - loss: 1.6820 - accuracy: 0.6230 - val_loss: 1.4763 - val_accuracy: 0.8020
Epoch 6/10
469/469 [==============================] - 61s 130ms/step - loss: 1.4528 - accuracy: 0.6639 - val_loss: 1.2174 - val_accuracy: 0.8135
Epoch 7/10
469/469 [==============================] - 61s 130ms/step - loss: 1.2414 - accuracy: 0.6962 - val_loss: 1.0017 - val_ac

Evaluation
- 10,000 images in our dataset will be evaluated to see how well our model functions.
- Excpected Accuracy for this project is 99% since the MNIST dataset is well-balanced. 

In [9]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.6497976779937744
Test accuracy: 0.854200005531311


Graphical User Interface for Predicting Digits
- For the GUI, we built a window that will navigate the canvas to draw digits.
- Using the predict_digit() function, it will take image as input and then utilize the trained model to indentify the digit. 

In [ ]:
from keras.models import load_model
from tkinter import *
import tkinter as tk
import win32gui
from PIL import ImageGrab, Image
import numpy as np
model = load_model('mnist.h5')
def predict_digit(img):
    #resize image to 28x28 pixels
    img = img.resize((28,28))
    #convert rgb to grayscale
    img = img.convert('L')
    img = np.array(img)
    #reshaping to support our model input and normalizing
    img = img.reshape(1,28,28,1)
    img = img/255.0
    #predicting the class
    res = model.predict([img])[0]
    return np.argmax(res), max(res)
class App(tk.Tk):
    def __init__(self):
        tk.Tk.__init__(self)
        self.x = self.y = 0
        # Creating elements
        self.canvas = tk.Canvas(self, width=300, height=300, bg = "white", cursor="cross")
        self.label = tk.Label(self, text="Thinking..", font=("Helvetica", 48))
        self.classify_btn = tk.Button(self, text = "Recognise", command =         self.classify_handwriting) 
        self.button_clear = tk.Button(self, text = "Clear", command = self.clear_all)
        # Grid structure
        self.canvas.grid(row=0, column=0, pady=2, sticky=W, )
        self.label.grid(row=0, column=1,pady=2, padx=2)
        self.classify_btn.grid(row=1, column=1, pady=2, padx=2)
        self.button_clear.grid(row=1, column=0, pady=2)
        #self.canvas.bind("<Motion>", self.start_pos)
        self.canvas.bind("<B1-Motion>", self.draw_lines)
    def clear_all(self):
        self.canvas.delete("all")
    def classify_handwriting(self):
        HWND = self.canvas.winfo_id() # get the handle of the canvas
        rect = win32gui.GetWindowRect(HWND) # get the coordinate of the canvas
        im = ImageGrab.grab(rect)
        digit, acc = predict_digit(im)
        self.label.configure(text= str(digit)+', '+ str(int(acc*100))+'%')
    def draw_lines(self, event):
        self.x = event.x
        self.y = event.y
        r=8
        self.canvas.create_oval(self.x-r, self.y-r, self.x + r, self.y + r, fill='black')
app = App()
mainloop()

Summary

- In this project, we built a deep learning model focused on handwritten digit recognition app. For image classfication purposes, we developed and trained a convolutional neural networks, which works great for classifying images. Last but not least, for pratical use we built a Graphical User Interface (GUI), where we can draw a digit (e.g., 1) on the interactive canvas and it will indentify the drawn digit.

Code Sources:
- GitHub
- Stackoverflow